##Results for SMAP Signals (Tabel III and V)

In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.2  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True, dropout=self.dropout_probability)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





SMAP=[  'P-1', 'S-1', 'E-1', 'E-2','E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13',   'P-3',
     'A-2', 'A-4', 'G-2',
    'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'G-4', 'T-3',  'D-12', 'B-1',  'G-7', 'P-7',
     'A-5', 'A-7', 'D-13',  'A-9']

MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']

#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)


avg=[]
train_signals=SMAP
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()

df_label=df_label[df_label['chan_id'].isin(SMAP)]

df_label=df_label.reset_index(drop=True)

for name in SMAP:

        label_row=df_label[df_label.chan_id==name]
       # pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]

        testing_predictions = bayesian_lstm.predict(X_test)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))




        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)


recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
print('precision_final', precision_final)
print('recall_final', recall_final)
print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  0.012808962725102901


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:463: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.37563272513243084
precision 0.9952208731593903 Signal P-1
recall 0.9117264229085316 Signal P-1
Accuracy 0.90759270158917 Signal P-1
F1 0.9516457728648181 Signal P-1
epoch 10 loss:  0.09938205033540726


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:463: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.1441059964485726
precision 0.8761461213797118 Signal S-1
recall 0.9401842886147118 Signal S-1
Accuracy 0.8311706051085918 Signal S-1
F1 0.9070363115865602 Signal S-1
epoch 10 loss:  0.052251316606998444


...

Proportion of points contained within 99% confidence interval: 0.4131771256234776
precision 0.9850434223222901 Signal A-7
recall 0.720418725005881 Signal A-7
Accuracy 0.7132583227003828 Signal A-7
F1 0.8322010869565217 Signal A-7
epoch 10 loss:  1.6175613382074516e-06


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:463: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.5733699202926956
precision 0.6587449933244326 Signal D-13
recall 0.9684003925417075 Signal D-13
Accuracy 0.6447144910492617 Signal D-13
F1 0.7841080651569329 Signal D-13
epoch 10 loss:  0.1616842895746231


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:463: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.3409306742640076
precision 0.8773315777924073 Signal A-9
recall 0.5084573318071983 Signal A-9
Accuracy 0.4745963912630579 Signal A-9
F1 0.6438003220611916 Signal A-9
precision_final 0.821206172698235
recall_final 0.8689576028952897
Accuracy_final 0.7491587624489744
F1_final 0.8249843402392615


##Fig.2 & Fig.4

In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.5  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability
                             )
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





SMAP=[  'P-1']

MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']

#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)


avg=[]
train_signals=SMAP
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()

df_label=df_label[df_label['chan_id'].isin(SMAP)]

df_label=df_label.reset_index(drop=True)

for name in SMAP:

        label_row=df_label[df_label.chan_id==name]
       # pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 100
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]

        testing_predictions = bayesian_lstm.predict(X_test)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))




        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]
        pdb.set_trace()

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)


recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
print('precision_final', precision_final)
print('recall_final', recall_final)
print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  0.010208198800683022
epoch 20 loss:  0.009512744843959808
epoch 30 loss:  0.008096105419099331
epoch 40 loss:  0.008220812305808067
epoch 50 loss:  0.008930835872888565
epoch 60 loss:  0.009708242490887642
epoch 70 loss:  0.006938711274415255
epoch 80 loss:  0.0054327272810041904
epoch 90 loss:  0.006684024818241596
epoch 100 loss:  0.005196703132241964


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:458: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.662978222483814
> <ipython-input-62-98144c616cc0>(596)<module>()->None
-> precision1 = tp / (tp + fp)
--KeyboardInterrupt--
(Pdb) exit


BdbQuit: ignored

##Results For MSL Signals





##Fig3 &5

In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.5  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





#SMAP=[  'P-1']

MSL=[
    'F-7']

#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)


avg=[]
train_signals=MSL
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()

df_label=df_label[df_label['chan_id'].isin(MSL)]

df_label=df_label.reset_index(drop=True)

for name in MSL:

        label_row=df_label[df_label.chan_id==name]
       # pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 100
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]

        testing_predictions = bayesian_lstm.predict(X_test)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))
        pdb.set_trace()




        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)

pdb.set_trace()
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
print('precision_final', precision_final)
print('recall_final', recall_final)
print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  0.007084657438099384
epoch 20 loss:  0.0069793774746358395
epoch 30 loss:  0.008323078975081444
epoch 40 loss:  0.007426393683999777
epoch 50 loss:  0.007448135409504175
epoch 60 loss:  0.006837795022875071
epoch 70 loss:  0.007651633117347956
epoch 80 loss:  0.007996022701263428
epoch 90 loss:  0.007092512212693691
epoch 100 loss:  0.006272081285715103


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:456: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



Proportion of points contained within 99% confidence interval: 0.46570182394924664
> <ipython-input-6-7adf8faf8451>(547)<module>()
-> realanomaly = label_data['index']
(Pdb) exit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.2  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





SMAP=[  'P-1', 'S-1', 'E-1', 'E-2','E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13',   'P-3',
     'A-2', 'A-4', 'G-2',
    'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'G-4', 'T-3',  'D-12', 'B-1',  'G-7', 'P-7',
     'A-5', 'A-7', 'D-13',  'A-9']


MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']
MSL=[
    'M-1', 'M-2', 'S-2',  'T-4', 'T-5', 'F-7', 'M-3', 'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-5', 'D-14',
    'T-9', 'T-8',  'D-15',  'M-7', 'F-8']

#SMAP=pd.Series(SMAP)
MSL=pd.Series(MSL)


avg=[]
train_signals=MSL
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()

df_label=df_label[df_label['chan_id'].isin(MSL)]

df_label=df_label.reset_index(drop=True)

for name in MSL:

        label_row=df_label[df_label.chan_id==name]
       # pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]

        testing_predictions = bayesian_lstm.predict(X_test)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5 *  test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))




        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)


recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
print('precision_final', precision_final)
print('recall_final', recall_final)
print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  0.03450700640678406


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.036612262902514334
precision 0.35231316725978645 Signal M-1
recall 0.8979591836734694 Signal M-1
Accuracy 0.6581385090427878 Signal M-1
F1 0.5060702875399361 Signal M-1
epoch 10 loss:  0.14075784385204315


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.04631671812968681
precision 0.22864768683274023 Signal M-2
recall 0.3899848254931715 Signal M-2
Accuracy 0.43934715483017206 Signal M-2
F1 0.2882781828379137 Signal M-2
epoch 10 loss:  1.3741993825533427e-05


...

Proportion of points contained within 99% confidence interval: 0.45344506517690875
precision 0.9787375415282392 Signal D-15
recall 0.6967833491012299 Signal D-15
Accuracy 0.6857541899441341 Signal D-15
F1 0.8140370268029843 Signal D-15
epoch 10 loss:  5.471200438478263e-06


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.32246039142590865
precision 0.9818893783651493 Signal M-7
recall 0.9520645467489322 Signal M-7
Accuracy 0.934762348555452 Signal M-7
F1 0.9667469879518072 Signal M-7
epoch 10 loss:  0.06680078059434891


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.29794105773112634
precision 0.9767801857585139 Signal F-8
recall 0.780618556701031 Signal F-8
Accuracy 0.766249495357287 Signal F-8
F1 0.867751547100619 Signal F-8
precision_final 0.7291771500131172
recall_final 0.8734828457892584
Accuracy_final 0.6959629451234786
F1_final 0.7416430723557341
